In [1]:
from fyenn_class import  pd_loaddata, pd, np 
path = './data_down/'
os.listdir(path)


['huawei_daily.csv',
 'huawei_output.csv',
 '~$入湖数据关系.xlsx',
 '入湖数据关系.csv',
 '入湖数据关系.xlsx']

In [100]:
%%time
huawei_output = pd_loaddata.pd_sep001(path + 'huawei_output.csv')
# huawei_daily = pd_loaddata.pd_sep001(path + 'huawei_daily.csv')
rel = pd_loaddata.pd_excel(path + '入湖数据关系', 2)
# huawei_output.shape
pd_loaddata.show_line(15, None)
rel = rel.drop(['部门', '站点'],axis = 1).drop_duplicates();rel

Wall time: 310 ms


,序号,对应现有看板业务量指标,对应效率指标,OU（成本中心）
0,1,按条收货-原材料,inbound,HUAWEDHW4S
3,2,按条发货-原材料,outbound,HUAWEDHW4S
23,3,PSN-原材料,labelling,HUAWEDHW4S
24,4,按条收货-荣耀,inbound,HONORDGHMS
26,5,按条发货-荣耀,outbound,HONORDGHMS
36,6,PSN-荣耀,labelling,HONORDGHMS
37,7,珠三角仓配趟数,transport,HUAWEDHWTS
39,8,贵阳仓配趟数,transport,HUAWEDGTRD
41,9,成品收货-T园区,inbound,HUAWEDGLSS
42,10,成品发货-T园区,outbound,HUAWEDGLSS


In [101]:
def datetime_(coach, col):
    """
    detail time to date; 
    split and assign to 3 cols of year m d
    """
    coach[col] = coach[col].astype(str).str.slice(0,10)
    coach = pd.concat([coach, pd.DataFrame(list(coach[col].str.split('-')))], axis =1)
    coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    coach = coach.rename({0:'year', 1:'month', 2: 'date'}, axis=1)
    return coach
huawei_output = datetime_(coach=huawei_output, col='update_date').drop(['createtime', 'updatetime'], axis = 1)
huawei_output = huawei_output.drop(['id','appid','entryid'], axis = 1).drop_duplicates().sort_values('update_date')
def search_col(df, str):
    """
    列名正则搜索.
    """
    return list(pd.Series(df.columns)[pd.Series(df.columns).str.match(str)])

In [102]:
# 识别异常列名
tran_col = list(pd.Series(huawei_output.columns)[
    pd.Series(huawei_output.columns).str.contains(
        '(receive|psn|send|transport_times|update_date|year|month|date|id)'
        ) == False
        ])

tran_col
# clean , case, withdraw)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_6504/2230356562.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pd.Series(huawei_output.columns).str.contains(


['t_product_clean', 'anshi_case', 'pre_withdraw']

In [159]:
# re = 'pear'
# pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna()

In [140]:
"""
ou 和正则匹配
"""
relist = ['hon', 'origi', 'pearl', 'guiy', 't_', 'r4_', 'nanh', 'ansh']
oulist = list(rel['OU（成本中心）'].unique())
del dict 
dict = dict(zip(relist, oulist))


In [143]:
def concat_(re, ou):
    huawei_output[search_col(huawei_output, re)].shape
    # n = 4 - huawei_output[search_col(huawei_output, re)].shape[1]
    # m = huawei_output[search_col(huawei_output, re)].shape[0]
    # print(m, n)
    data = pd.concat(
        [
            huawei_output[search_col(huawei_output, re)], 
            # pd.DataFrame(np.zeros(shape=(m, n), dtype=int)),
            huawei_output[['update_date', 'year', 'month', 'date']]
        ]
        , axis = 1).sort_values('update_date')
    # print(data.shape)
    data.columns = list(pd.Series(data.columns).str.extract(
        '(receive|psn|send|transport_times|update_date|year|month|date)'
        )[0])
    try:
        data = data.rename({np.nan:'addition'}, axis = 1)
    except:
        pass
    data['ou'] = ou
    data['station'] = pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna().iloc[0,0]
    data['addition_type'] = pd.Series(tran_col).str.extract("(" + re + ".+)")[0][0]

    return data.reset_index(drop=True)

df = pd.DataFrame()
for re in dict:
    print(re, dict[re])
    df = pd.concat([df, concat_(re, dict[re])], axis = 0)
df = df.fillna(0)

hon HUAWEDHW4S
origi HONORDGHMS
pearl HUAWEDHWTS
guiy HUAWEDGTRD
t_ HUAWEDGLSS
r4_ HUAWEDHW1S
nanh HUAWEDGNHS
ansh NEXPEDGIHS


In [158]:
df.query("year == '2021' & month == '05' & date == '29'")

,receive,send,psn,transport_times,update_date,year,month,date,ou,station,addition_type,addition
149,151.0,34.0,0.0,0.0,2021-05-29,2021,05,29,HUAWEDHW4S,honor,0,0.0
149,625.0,969.0,0.0,0.0,2021-05-29,2021,05,29,HONORDGHMS,origin,0,0.0
149,0.0,0.0,0.0,0.0,2021-05-29,2021,05,29,HUAWEDHWTS,pearlriver,0,0.0
149,0.0,0.0,0.0,0.0,2021-05-29,2021,05,29,HUAWEDGTRD,guiyang,0,0.0
149,13390.0,130.0,0.0,0.0,2021-05-29,2021,05,29,HUAWEDGLSS,t_times,t_product_clean,0.0
149,0.0,77.0,92.0,0.0,2021-05-29,2021,05,29,HUAWEDHW1S,r4_receive,0,0.0
149,175.0,1548.0,1163.0,0.0,2021-05-29,2021,05,29,HUAWEDGNHS,nanhua,0,0.0
149,0.0,0.0,0.0,0.0,2021-05-29,2021,05,29,NEXPEDGIHS,anshi,0,0.0


In [130]:
list(pd.Series(huawei_output.columns).str.extract("(" + 'pearl' + "[a-z]+)").dropna()[0])[0]

'pearlriver'

In [28]:
print("(" + 'hornor' + "[az])")

(hornor[az])
